In [1]:
import geopandas as gpd
import numpy as np
import osmnx as ox
from scipy.stats import gaussian_kde
import folium
from folium.plugins import HeatMap
from shapely.geometry import box

In [2]:
# Step 1: Get Berlin Boundary
berlin_boundary = ox.geocode_to_gdf("Berlin, Germany")

In [3]:
gdf = gpd.read_file('purchases.geojson')  # Replace with your GeoJSON file path

In [4]:
gdf = gdf[gdf['BruttoSumme'].notnull() & (gdf['BruttoSumme'] > 0)]

In [5]:
gdf = gdf.to_crs("EPSG:4326")  # Reproject to Lat/Lon for Folium
berlin_boundary = berlin_boundary.to_crs("EPSG:4326")

In [6]:
# Step 3: Extract coordinates and weights for KDE
coords = np.vstack([gdf.geometry.x, gdf.geometry.y])
weights = gdf['BruttoSumme']  # Replace with the correct column name

In [7]:
# Ensure no NaN or inf values
valid_mask = np.isfinite(coords).all(axis=0) & np.isfinite(weights)
coords = coords[:, valid_mask]
weights = weights[valid_mask]

In [8]:
# Step 5: Perform KDE
kde = gaussian_kde(coords, weights = weights)
xmin, ymin, xmax, ymax = berlin_boundary.total_bounds
X, Y = np.meshgrid(np.linspace(xmin, xmax, 500), np.linspace(ymin, ymax, 500))
positions = np.vstack([X.ravel(), Y.ravel()])
Z = np.reshape(kde(positions).T, X.shape)

In [9]:
# Step 4: Convert KDE Grid to Heatmap Format
# Prepare data for HeatMap: list of (latitude, longitude, intensity)
heat_data = []
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        heat_data.append([Y[i, j], X[i, j], Z[i, j]])

In [10]:
# Step 5: Create Folium Map
# Center map at Berlin's mean coordinates
berlin_center = [gdf.geometry.y.mean(), gdf.geometry.x.mean()]
m = folium.Map(location=berlin_center, zoom_start=12, tiles="CartoDB Positron")

# Add HeatMap layer
HeatMap(heat_data, min_opacity=0.2, max_opacity=0.8, radius=15, blur=10).add_to(m)

In [11]:
# Add Berlin boundary
folium.GeoJson(berlin_boundary, name="Berlin Boundary", style_function=lambda x: {
    "fillOpacity": 0.1,
    "weight": 1,
    "color": "black"
}).add_to(m)

# Step 6: Save and Display
m.save("berlin_kde_heatmap.html")
m


In [ ]:



# Create a grid around Berlin boundaries
xmin, ymin, xmax, ymax = berlin_boundary.total_bounds
X, Y = np.meshgrid(np.linspace(xmin, xmax, 200), np.linspace(ymin, ymax, 200))  # 200x200 grid
positions = np.vstack([X.ravel(), Y.ravel()])

# Perform KDE
kde = KernelDensity(kernel='gaussian', bandwidth=0.01)  # Adjust bandwidth for density smoothness
kde.fit(coords.T, sample_weight=weights)

Z = np.exp(kde.score_samples(positions.T)).reshape(X.shape)

# Step 4: Convert KDE Grid to Heatmap Format
# Prepare data for HeatMap: list of (latitude, longitude, intensity)
heat_data = []
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        heat_data.append([Y[i, j], X[i, j], Z[i, j]])

# Step 5: Create Folium Map
# Center map at Berlin's mean coordinates
berlin_center = [gdf.geometry.y.mean(), gdf.geometry.x.mean()]
m = folium.Map(location=berlin_center, zoom_start=12, tiles="CartoDB Positron")

# Add HeatMap layer
HeatMap(heat_data, min_opacity=0.2, max_opacity=0.8, radius=15, blur=10).add_to(m)

# Add Berlin boundary
folium.GeoJson(berlin_boundary, name="Berlin Boundary", style_function=lambda x: {
    "fillOpacity": 0.1,
    "weight": 1,
    "color": "black"
}).add_to(m)

# Step 6: Save and Display
m.save("berlin_kde_heatmap.html")
m
